# Lab 2: Detecting Bad Sensors in Power System Monitoring

In this lab, our goal is to detect bad sensor data measured on the IEEE 14 bus test
system shown below. The power flow equations that couple the voltages and power flows are 
nonlinear in nature, as discussed in class. We will load the sensor data from the
file 'sensorData14Bus.csv', and utilize SVM to perform the bad data detection.
We aim to understand how various parameters such as the nature of the corrupt data,
the number of corrupt data, etc., affect our abilities to classify the data.

<img src="IEEE14bus.png">

### First, we need to call the needed libraries

In [146]:
import numpy as np
import os
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from IPython.display import Image
import pandas as pd


### Loading the data 

Load the sensor data from the IEEE 14 bus test system, that has 14 buses
 and 20 branches. The data has been generated by adding a small noise
 to feasible voltages and power flows.
     
     Columns 1-14 contain bus voltage magnitudes.
     
     Columns 15-28 contain bus voltage phase angles.
     
     Columns 29-48 contain real power flow on all branches.
     
     Columns 49-68 contain reactive power flow on all branches.

In [147]:
nBuses = 14
nBranches = 20

# Select the bus numbers you monitor. For convenience, we have selected it for you.
# The '-1' makes them columns as per Python's convention of starting to number
# from 0.
busesToSample = np.array([1, 2, 5, 10, 13]) - 1
columnsForBuses = np.concatenate((busesToSample, busesToSample + 14))

# Select the branches that you monitor.
branchesToSample = np.array([1, 3, 5, 10, 11, 15, 17, 20]) - 1
columnsForBranches = np.concatenate((branchesToSample + 28,
                                     branchesToSample + 48))

# Load the sensor data from the file 'sensorData14Bus.csv' in 'X' from the columns
# specified in 'columnsForBuses' and 'columnsForBranches'. The csv file is comma
# separated. Read a maximum of 5000 lines. Make sure your data is a numpy array
# with each column typecast as 'np.float32'.
X = np.genfromtxt('sensorData14Bus.csv', dtype=np.float32, delimiter=',',
                  usecols=np.concatenate((columnsForBuses, columnsForBranches)),
                  max_rows=5000)

nDataPoints = np.shape(X)[0]
nFeatures = np.shape(X)[1]

print("Loaded sensor data on IEEE 14 bus system.")
print("Number of data points = %d, number of features = %d"
      % (nDataPoints, nFeatures))

Loaded sensor data on IEEE 14 bus system.
Number of data points = 5000, number of features = 26


### Curroption Models 

Intentionally corrupt the first 'nCorrupt' rows of the data by adding
 a quantity to one or two sensor measurements that is not representative of
 our error model. We aim to study what nature of corruption is easier
 or difficult to detect.
 Specifically, we shall study 3 different models:
 
     1. 'corruptionModel' = 1 : Add a random number with a bias to one of the measurements.
     
     2. 'corruptionModel' = 2 : Add a random number without bias to one of the measurements.
     
     3. 'corruptionModel' = 3 : Add a random number with a bias to both the measurements.
     
In all these cases, we will multiply the sensor data by either a uniform or a normal random number multiplied by 'multiplicationFactor'.


In [148]:
# Choose a corruption model.
nCorrupt = int(nDataPoints/3)
corruptionModel = 1
multiplicationFactor = 0.5

# Choose which data to tamper with, that can be a voltage magnitude,
# voltage phase angle, real power flow on a branch, reactive power flow
# on a branch. We create functions to extract the relevant column to
# corrupt the corresponding data in the 'ii'-th bus or branch.
voltageMagnitudeColumn = lambda ii: ii

voltageAngleColumn = lambda ii: ii + np.shape(busesToSample)[0]

realPowerColumn = lambda ii: ii + 2*np.shape(busesToSample)[0]
reactivePowerColumn = lambda ii: ii + 2*np.shape(busesToSample)[0] + np.shape(branchesToSample)[0]

# Encode two different kinds of columns to corrupt.
# Option 1: Corrupt real power columns only.
# Option 2: Corrupt real power and voltage magnitude.
columnsToCorruptOption = 2

if columnsToCorruptOption == 1:
    columnsToCorrupt = [realPowerColumn(1),
                        realPowerColumn(2)]
else:
    columnsToCorrupt = [voltageMagnitudeColumn(0),
                        realPowerColumn(1)]

# Corrupt the data appropriately, given the options.
for index in range(nCorrupt):

    if corruptionModel == 1:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
    elif corruptionModel == 2:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.randn())
    else:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
        X[index, columnsToCorrupt[1]] \
            *= (1 + multiplicationFactor * np.random.rand())


 It is always a good practice to scale your data to run SVM. Notice that we are
 cheating a little when we scale the entire data set 'X', because our training and
 test sets are derived from 'X'. Ideally, one would have to scale the training
 and test sets separately. Create the appropriate labels and shuffle the lists 'X' and 'Y' together.


In [149]:

X = preprocessing.StandardScaler().fit_transform(X)

# Create the labels as a column of 1's for the first 'nCorrupt' rows, and
# 0's for the rest.
Y = np.concatenate((np.ones(nCorrupt), np.zeros(nDataPoints-nCorrupt)))


# Shuffle the features and the labels together.
XY = list(zip(X, Y))
np.random.shuffle(XY)
X, Y = zip(*XY)


Recall from the first lab that 'test_size' determines what fraction of the data becomes your test set.

## Task 1 (10 points)

Split the dataset into two parts: training and testing.
Store the training set in the variables 'trainX' and 'trainY'.
 Store the testing set in the variables 'testX' and 'testY.
 Reserve 20% of the data for testing.
The function 'train_test_split' may prove useful.


In [150]:
# Enter your code here
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2)

## Task 2 (10 points)

 Define the support vector machine classifier and train on the variables 'trainX' and 'trainY'. Use the SVC library from sklearn.svm. Only specify three hyper-parameters: 'kernel', 'degree', and 'max_iter'. Limit the maximum number of iterations to 100000 at the most. Set the kernel to be a linear classifier first. You may have to change it to report the results with other kernels. The parameter 'degree' specifies the degree for polynomial kernels. This parameter is not used for other kernels. The functions 'svm.SVC' and 'fit' will prove useful.



In [151]:
# Enter your code here
model = svm.SVC(kernel = 'linear', degree= 1, max_iter= 1000)
model.fit(trainX,trainY)

C:\Users\chazp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVC(degree=1, kernel='linear', max_iter=1000)

## Task 3 (10 points)

Predict the labels on the 'testX' dataset and store them in 'predictY'.


In [152]:
# Enter your code here
predictY = model.predict(testX)

## Task 4 (10 points)

Print the 'classification_report' to see how well 'predictY' matches with 'testY'.


In [153]:
# Enter your code here
print(classification_report(testY, predictY))

              precision    recall  f1-score   support

         0.0       0.98      0.97      0.97       675
         1.0       0.94      0.95      0.94       325

    accuracy                           0.96      1000
   macro avg       0.96      0.96      0.96      1000
weighted avg       0.96      0.96      0.96      1000



Print svm's internal accuracy score as a percentage.

In [154]:
# Enter your code here
print("Accuracy score of SVM = %1.2f" % model.score(testX, testY))

Accuracy score of SVM = 0.96


## Task 5

We would like to compare 'classification_report' with this score for various runs. Let us consider the following cases: 

### Case 1:

Only have sensor measurements from the first 5 branches. Choose option 1 in the 'columnsToCorruptOption'. Examine how well linear kernels perform when 'corruptionModel' = 1, 'corruptionModel' = 2, and 'corruptionModel'= 3. In case linear kernels do not perform well, you may try 'rbf' or polynomial kernels with degree 2.

### Case 2:

Choose 'corruptionModel = 1' with 'linear' kernel. Does it pay to monitor voltage magnitudes than power flows? In other words, do you consistently get better results when you choose 'columnsToCorruptOption' as 2? Make these judgements using the average score of at least 5 runs.


#### Your task is to investigate the above two cases. You may add a few 'Markdown' and 'Code' cells below with your comments, code, and results. You can also report your results as a pandas DataFrame. You are free to report your results in your own way.

### Case 1:

In [155]:
nBuses = 14
nBranches = 20

# Select the bus numbers you monitor. For convenience, we have selected it for you.
# The '-1' makes them columns as per Python's convention of starting to number
# from 0.
busesToSample = np.array([1, 2, 5, 10, 13]) - 1
columnsForBuses = np.concatenate((busesToSample, busesToSample + 14))

# Select the branches that you monitor.
branchesToSample = np.array([1, 5, 11, 17, 20]) - 1
columnsForBranches = np.concatenate((branchesToSample + 28,
                                     branchesToSample + 48))

# Load the sensor data from the file 'sensorData14Bus.csv' in 'X' from the columns
# specified in 'columnsForBuses' and 'columnsForBranches'. The csv file is comma
# separated. Read a maximum of 5000 lines. Make sure your data is a numpy array
# with each column typecast as 'np.float32'.
X = np.genfromtxt('sensorData14Bus.csv', dtype=np.float32, delimiter=',',
                  usecols=np.concatenate((columnsForBuses, columnsForBranches)),
                  max_rows=5000)

nDataPoints = np.shape(X)[0]
nFeatures = np.shape(X)[1]
 
print("Loaded sensor data on IEEE 14 bus system.")
print("Number of data points = %d, number of features = %d"
      % (nDataPoints, nFeatures))
# Choose a corruption model.
nCorrupt = int(nDataPoints/3)
corruptionModel = 1
multiplicationFactor = 0.5

# Choose which data to tamper with, that can be a voltage magnitude,
# voltage phase angle, real power flow on a branch, reactive power flow
# on a branch. We create functions to extract the relevant column to
# corrupt the corresponding data in the 'ii'-th bus or branch.
voltageMagnitudeColumn = lambda ii: ii

voltageAngleColumn = lambda ii: ii + np.shape(busesToSample)[0]

realPowerColumn = lambda ii: ii + 2*np.shape(busesToSample)[0]
reactivePowerColumn = lambda ii: ii + 2*np.shape(busesToSample)[0] + np.shape(branchesToSample)[0]

# Encode two different kinds of columns to corrupt.
# Option 1: Corrupt real power columns only.
# Option 2: Corrupt real power and voltage magnitude.
columnsToCorruptOption = 1

if columnsToCorruptOption == 1:
    columnsToCorrupt = [realPowerColumn(1),
                        realPowerColumn(2)]
else:
    columnsToCorrupt = [voltageMagnitudeColumn(0),
                        realPowerColumn(1)]

# Corrupt the data appropriately, given the options.
for index in range(nCorrupt):

    if corruptionModel == 1:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
    elif corruptionModel == 2:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.randn())
    else:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
        X[index, columnsToCorrupt[1]] \
            *= (1 + multiplicationFactor * np.random.rand())


X = preprocessing.StandardScaler().fit_transform(X)

# Create the labels as a column of 1's for the first 'nCorrupt' rows, and
# 0's for the rest.
Y = np.concatenate((np.ones(nCorrupt), np.zeros(nDataPoints-nCorrupt)))


# Shuffle the features and the labels together.
XY = list(zip(X, Y))
np.random.shuffle(XY)
X, Y = zip(*XY)

trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2)
model = svm.SVC(kernel = 'linear', degree= 1, max_iter= 1000)
model.fit(trainX,trainY)
predictY = model.predict(testX)
print(classification_report(testY, predictY))
print("Accuracy score of SVM = %1.2f" % model.score(testX, testY))

Loaded sensor data on IEEE 14 bus system.
Number of data points = 5000, number of features = 20
              precision    recall  f1-score   support

         0.0       0.96      0.92      0.94       676
         1.0       0.85      0.92      0.88       324

    accuracy                           0.92      1000
   macro avg       0.90      0.92      0.91      1000
weighted avg       0.92      0.92      0.92      1000

Accuracy score of SVM = 0.92


C:\Users\chazp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [156]:
nBuses = 14
nBranches = 20

# Select the bus numbers you monitor. For convenience, we have selected it for you.
# The '-1' makes them columns as per Python's convention of starting to number
# from 0.
busesToSample = np.array([1, 2, 5, 10, 13]) - 1
columnsForBuses = np.concatenate((busesToSample, busesToSample + 14))

# Select the branches that you monitor.
branchesToSample = np.array([1, 5, 11, 17, 20]) - 1
columnsForBranches = np.concatenate((branchesToSample + 28,
                                     branchesToSample + 48))

# Load the sensor data from the file 'sensorData14Bus.csv' in 'X' from the columns
# specified in 'columnsForBuses' and 'columnsForBranches'. The csv file is comma
# separated. Read a maximum of 5000 lines. Make sure your data is a numpy array
# with each column typecast as 'np.float32'.
X = np.genfromtxt('sensorData14Bus.csv', dtype=np.float32, delimiter=',',
                  usecols=np.concatenate((columnsForBuses, columnsForBranches)),
                  max_rows=5000)

nDataPoints = np.shape(X)[0]
nFeatures = np.shape(X)[1]
 
print("Loaded sensor data on IEEE 14 bus system.")
print("Number of data points = %d, number of features = %d"
      % (nDataPoints, nFeatures))
# Choose a corruption model.
nCorrupt = int(nDataPoints/3)
corruptionModel = 2
multiplicationFactor = 0.5

# Choose which data to tamper with, that can be a voltage magnitude,
# voltage phase angle, real power flow on a branch, reactive power flow
# on a branch. We create functions to extract the relevant column to
# corrupt the corresponding data in the 'ii'-th bus or branch.
voltageMagnitudeColumn = lambda ii: ii

voltageAngleColumn = lambda ii: ii + np.shape(busesToSample)[0]

realPowerColumn = lambda ii: ii + 2*np.shape(busesToSample)[0]
reactivePowerColumn = lambda ii: ii + 2*np.shape(busesToSample)[0] + np.shape(branchesToSample)[0]

# Encode two different kinds of columns to corrupt.
# Option 1: Corrupt real power columns only.
# Option 2: Corrupt real power and voltage magnitude.
columnsToCorruptOption = 1

if columnsToCorruptOption == 1:
    columnsToCorrupt = [realPowerColumn(1),
                        realPowerColumn(2)]
else:
    columnsToCorrupt = [voltageMagnitudeColumn(0),
                        realPowerColumn(1)]

# Corrupt the data appropriately, given the options.
for index in range(nCorrupt):

    if corruptionModel == 1:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
    elif corruptionModel == 2:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.randn())
    else:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
        X[index, columnsToCorrupt[1]] \
            *= (1 + multiplicationFactor * np.random.rand())



X = preprocessing.StandardScaler().fit_transform(X)

# Create the labels as a column of 1's for the first 'nCorrupt' rows, and
# 0's for the rest.
Y = np.concatenate((np.ones(nCorrupt), np.zeros(nDataPoints-nCorrupt)))


# Shuffle the features and the labels together.
XY = list(zip(X, Y))
np.random.shuffle(XY)
X, Y = zip(*XY)

trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2)
model = svm.SVC(kernel = 'rbf', degree= 2, max_iter= 1000)
model.fit(trainX,trainY)
predictY = model.predict(testX)
print(classification_report(testY, predictY))
print("Accuracy score of SVM = %1.2f" % model.score(testX, testY))

Loaded sensor data on IEEE 14 bus system.
Number of data points = 5000, number of features = 20


C:\Users\chazp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.81      1.00      0.90       673
         1.0       0.99      0.53      0.69       327

    accuracy                           0.85      1000
   macro avg       0.90      0.77      0.80      1000
weighted avg       0.87      0.85      0.83      1000

Accuracy score of SVM = 0.85


In [157]:
nBuses = 14
nBranches = 20

# Select the bus numbers you monitor. For convenience, we have selected it for you.
# The '-1' makes them columns as per Python's convention of starting to number
# from 0.
busesToSample = np.array([1, 2, 5, 10, 13]) - 1
columnsForBuses = np.concatenate((busesToSample, busesToSample + 14))

# Select the branches that you monitor.
branchesToSample = np.array([1, 5, 11,17, 20]) - 1
columnsForBranches = np.concatenate((branchesToSample + 28,
                                     branchesToSample + 48))

# Load the sensor data from the file 'sensorData14Bus.csv' in 'X' from the columns
# specified in 'columnsForBuses' and 'columnsForBranches'. The csv file is comma
# separated. Read a maximum of 5000 lines. Make sure your data is a numpy array
# with each column typecast as 'np.float32'.
X = np.genfromtxt('sensorData14Bus.csv', dtype=np.float32, delimiter=',',
                  usecols=np.concatenate((columnsForBuses, columnsForBranches)),
                  max_rows=5000)

nDataPoints = np.shape(X)[0]
nFeatures = np.shape(X)[1]
 
print("Loaded sensor data on IEEE 14 bus system.")
print("Number of data points = %d, number of features = %d"
      % (nDataPoints, nFeatures))
# Choose a corruption model.
nCorrupt = int(nDataPoints/3)
corruptionModel = 3
multiplicationFactor = 0.5

# Choose which data to tamper with, that can be a voltage magnitude,
# voltage phase angle, real power flow on a branch, reactive power flow
# on a branch. We create functions to extract the relevant column to
# corrupt the corresponding data in the 'ii'-th bus or branch.
voltageMagnitudeColumn = lambda ii: ii

voltageAngleColumn = lambda ii: ii + np.shape(busesToSample)[0]

realPowerColumn = lambda ii: ii + 2*np.shape(busesToSample)[0]
reactivePowerColumn = lambda ii: ii + 2*np.shape(busesToSample)[0] + np.shape(branchesToSample)[0]

# Encode two different kinds of columns to corrupt.
# Option 1: Corrupt real power columns only.
# Option 2: Corrupt real power and voltage magnitude.
columnsToCorruptOption = 1

if columnsToCorruptOption == 1:
    columnsToCorrupt = [realPowerColumn(1),
                        realPowerColumn(2)]
else:
    columnsToCorrupt = [voltageMagnitudeColumn(0),
                        realPowerColumn(1)]

# Corrupt the data appropriately, given the options.
for index in range(nCorrupt):

    if corruptionModel == 1:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
    elif corruptionModel == 2:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.randn())
    else:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
        X[index, columnsToCorrupt[1]] \
            *= (1 + multiplicationFactor * np.random.rand())


X = preprocessing.StandardScaler().fit_transform(X)

# Create the labels as a column of 1's for the first 'nCorrupt' rows, and
# 0's for the rest.
Y = np.concatenate((np.ones(nCorrupt), np.zeros(nDataPoints-nCorrupt)))


# Shuffle the features and the labels together.
XY = list(zip(X, Y))
np.random.shuffle(XY)
X, Y = zip(*XY)

trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2)
model = svm.SVC(kernel = 'linear', degree= 1, max_iter= 1000)
model.fit(trainX,trainY)
predictY = model.predict(testX)
print(classification_report(testY, predictY))
print("Accuracy score of SVM = %1.2f" % model.score(testX, testY))

Loaded sensor data on IEEE 14 bus system.
Number of data points = 5000, number of features = 20
              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98       675
         1.0       0.97      0.94      0.96       325

    accuracy                           0.97      1000
   macro avg       0.97      0.97      0.97      1000
weighted avg       0.97      0.97      0.97      1000

Accuracy score of SVM = 0.97


C:\Users\chazp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


##### Part 1 Answers

CorryptionModel #1 = For corruption model1 our training data is linearly seperable due to the fact that our random noise is biased to be only positive so a linear SVM seperates our data very well.  

CorruptionModel #2 = For corruption model2 our training data is no longer linearly seperable due to the fact our random noise can be either positive or negative. If we use linearSVM we are essentially guessing and get roughly 50% accuracy. If we use RBF as a kernel function it is much easier to seperate our data  

CorruptionModel #3 = For corruption model3 our training data is the most linearly seperable as we add biased noise to both of our features and because of this a linearSVM does best for this data.

### Case 2

In [158]:
def case2func(num):
    nBuses = 14
    nBranches = 20

    # Select the bus numbers you monitor. For convenience, we have selected it for you.
    # The '-1' makes them columns as per Python's convention of starting to number
    # from 0.
    busesToSample = np.array([1, 2, 5, 10, 13]) - 1
    columnsForBuses = np.concatenate((busesToSample, busesToSample + 14))

    # Select the branches that you monitor.
    branchesToSample = np.array([1, 5, 11, 17, 20]) - 1
    columnsForBranches = np.concatenate((branchesToSample + 28,
                                        branchesToSample + 48))

    # Load the sensor data from the file 'sensorData14Bus.csv' in 'X' from the columns
    # specified in 'columnsForBuses' and 'columnsForBranches'. The csv file is comma
    # separated. Read a maximum of 5000 lines. Make sure your data is a numpy array
    # with each column typecast as 'np.float32'.
    X = np.genfromtxt('sensorData14Bus.csv', dtype=np.float32, delimiter=',',
                    usecols=np.concatenate((columnsForBuses, columnsForBranches)),
                    max_rows=5000)

    nDataPoints = np.shape(X)[0]
    nFeatures = np.shape(X)[1]
    
    print("Loaded sensor data on IEEE 14 bus system.")
    print("Number of data points = %d, number of features = %d"
        % (nDataPoints, nFeatures))
    # Choose a corruption model.
    nCorrupt = int(nDataPoints/3)
    corruptionModel = 1
    multiplicationFactor = 0.5

    # Choose which data to tamper with, that can be a voltage magnitude,
    # voltage phase angle, real power flow on a branch, reactive power flow
    # on a branch. We create functions to extract the relevant column to
    # corrupt the corresponding data in the 'ii'-th bus or branch.
    voltageMagnitudeColumn = lambda ii: ii

    voltageAngleColumn = lambda ii: ii + np.shape(busesToSample)[0]

    realPowerColumn = lambda ii: ii + 2*np.shape(busesToSample)[0]
    reactivePowerColumn = lambda ii: ii + 2*np.shape(busesToSample)[0] + np.shape(branchesToSample)[0]

    # Encode two different kinds of columns to corrupt.
    # Option 1: Corrupt real power columns only.
    # Option 2: Corrupt real power and voltage magnitude.
    columnsToCorruptOption = num

    if columnsToCorruptOption == 1:
        columnsToCorrupt = [realPowerColumn(1),
                            realPowerColumn(2)]
    else:
        columnsToCorrupt = [voltageMagnitudeColumn(0),
                            realPowerColumn(1)]

    # Corrupt the data appropriately, given the options.
    for index in range(nCorrupt):

        if corruptionModel == 1:
            X[index, columnsToCorrupt[0]] \
                *= (1 + multiplicationFactor * np.random.rand())
        elif corruptionModel == 2:
            X[index, columnsToCorrupt[0]] \
                *= (1 + multiplicationFactor * np.random.randn())
        else:
            X[index, columnsToCorrupt[0]] \
                *= (1 + multiplicationFactor * np.random.rand())
            X[index, columnsToCorrupt[1]] \
                *= (1 + multiplicationFactor * np.random.rand())


    X = preprocessing.StandardScaler().fit_transform(X)

    # Create the labels as a column of 1's for the first 'nCorrupt' rows, and
    # 0's for the rest.
    Y = np.concatenate((np.ones(nCorrupt), np.zeros(nDataPoints-nCorrupt)))


    # Shuffle the features and the labels together.
    XY = list(zip(X, Y))
    np.random.shuffle(XY)
    X, Y = zip(*XY)

    trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2)
    model = svm.SVC(kernel = 'linear', degree= 1, max_iter= 1000)
    model.fit(trainX,trainY)
    predictY = model.predict(testX)
    temp = model.score(testX, testY)
    # print(classification_report(testY, predictY))
    # print("Accuracy score of SVM = %1.2f" % model.score(testX, testY))
    return temp

In [159]:
# print(case2func(1))
# print(case2func(2))
sum1 = 0
sum2 = 0
for i in range(5):
    sum1 += case2func(1)
    sum2 += case2func(2)
print("Columns to Corrupt 1 average accuracy: ", sum1/5)
print("Columns to Corrupt 2 average accuracy: ", sum2/5)

Loaded sensor data on IEEE 14 bus system.
Number of data points = 5000, number of features = 20


C:\Users\chazp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Loaded sensor data on IEEE 14 bus system.
Number of data points = 5000, number of features = 20


C:\Users\chazp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Loaded sensor data on IEEE 14 bus system.
Number of data points = 5000, number of features = 20


...

C:\Users\chazp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Loaded sensor data on IEEE 14 bus system.
Number of data points = 5000, number of features = 20


C:\Users\chazp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Loaded sensor data on IEEE 14 bus system.
Number of data points = 5000, number of features = 20
Columns to Corrupt 1 average accuracy:  0.9284000000000001
Columns to Corrupt 2 average accuracy:  0.9735999999999999


C:\Users\chazp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


When we base our SVM off of both the real power and the voltagemagnitude we are able to be more accurate in our classification due to how we cover more variance with 2 different variables then we do with 1. It is easier to linearly seperate data when we only corrupt 2d data in 1 dimension.